# Snowfakery MCP Playground

This notebook is a low-friction way to use the Snowfakery MCP server inside a Codespace.

What you can do here:
- Validate recipes
- Analyze recipes
- Run recipes and browse run artifacts

Under the hood, this notebook launches the MCP server via `uv run snowfakery-mcp` (stdio transport) and calls MCP tools using the Python MCP client.

In [17]:
from __future__ import annotations

import json
from pathlib import Path
from typing import Any

from mcp.client.session import ClientSession
from mcp.client.stdio import StdioServerParameters, stdio_client


def find_repo_root(start: Path | None = None) -> Path:
    start = (start or Path.cwd()).resolve()
    for p in [start, *start.parents]:
        if (p / 'pyproject.toml').exists():
            return p
    raise RuntimeError('Could not find repo root (pyproject.toml)')


REPO_ROOT = find_repo_root()
REPO_ROOT

PosixPath('/workspaces/snowfakery-mcp')

In [19]:
tools = await list_tools()
tools

['analyze_recipe',
 'generate_mapping',
 'get_example',
 'get_schema',
 'list_capabilities',
 'list_examples',
 'run_recipe',
 'search_docs',
 'validate_recipe']

## Validate + run a tiny recipe

In [20]:
recipe = '''
- snowfakery_version: 3
- object: Person
  count: 2
  fields:
    name: Buster
'''

validate = await call_tool('validate_recipe', {'recipe_text': recipe, 'strict_mode': True})
validate

{'valid': True, 'errors': []}

In [21]:
run = await call_tool(
    'run_recipe',
    {
        'recipe_text': recipe,
        'reps': 1,
        'output_format': 'txt',
        'capture_output': True,
        'strict_mode': True,
    },
)
run

{'run_id': '1151c70086364febb71f9ba33a6a23df',
 'ok': True,
 'output_format': 'txt',
 'stdout_text': 'Person(id=1, name=Buster)\nPerson(id=2, name=Buster)\n',
 'stdout_truncated': False,
 'resources': ['snowfakery://runs/1151c70086364febb71f9ba33a6a23df/output.txt'],
 'summary': '<snowfakery.data_generator.ExecutionSummary object at 0x7b2afcd6fb60>'}

## Analyze recipe structure

In [ ]:
analysis = await call_tool('analyze_recipe', {'recipe_text': recipe})
analysis